In [6]:
import os
import gc
import sys
import glob
import numpy as np
import pandas as pd
import netCDF4 as nc
from datetime import datetime, timedelta
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
import matplotlib.ticker as mticker

In [7]:
var_output    = pd.read_csv(f'/g/data/w97/mm3972/scripts/PLUMBER2/LSM_VPD_PLUMBER2/txt/Qle_all_sites.csv',na_values=[''])

In [9]:
var_output.columns

Index(['Unnamed: 0', 'time', 'model_1lin', '1lin_EF', 'model_3km27',
       '3km27_EF', 'model_6km729', '6km729_EF', 'model_6km729lag',
       '6km729lag_EF', 'model_ACASA', 'ACASA_EF', 'model_CABLE', 'CABLE_EF',
       'model_CABLE-POP-CN', 'CABLE-POP-CN_EF', 'model_CHTESSEL_ERA5_3',
       'CHTESSEL_ERA5_3_EF', 'model_CHTESSEL_Ref_exp1', 'CHTESSEL_Ref_exp1_EF',
       'model_CLM5a', 'CLM5a_EF', 'model_GFDL', 'GFDL_EF',
       'model_JULES_GL9_withLAI', 'JULES_GL9_withLAI_EF', 'model_JULES_test',
       'JULES_test_EF', 'model_LSTM_eb', 'LSTM_eb_EF', 'model_LSTM_raw',
       'LSTM_raw_EF', 'model_Manabe', 'Manabe_EF', 'model_ManabeV2',
       'ManabeV2_EF', 'model_MATSIRO', 'MATSIRO_EF', 'model_MuSICA',
       'MuSICA_EF', 'model_NASAEnt', 'NASAEnt_EF', 'model_NoahMPv401',
       'NoahMPv401_EF', 'model_ORC2_r6593', 'ORC2_r6593_EF',
       'model_ORC2_r6593_CO2', 'ORC2_r6593_CO2_EF', 'model_ORC3_r7245_NEE',
       'ORC3_r7245_NEE_EF', 'model_ORC3_r8120', 'ORC3_r8120_EF',
       'model